<a href="https://colab.research.google.com/github/Sivadogga007/Genny_IE643_Course_Project/blob/master/latex_code_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()# please mail 24m1528@iitb.ac.in for credentials. I will reply asap

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

doggasivakumar_gurgurov_model_im2latest_path = kagglehub.dataset_download('doggasivakumar/gurgurov-model-im2latest')
doggasivakumar_nadikcew_path = kagglehub.dataset_download('doggasivakumar/nadikcew')
doggasivakumar_smallocr_path = kagglehub.dataset_download('doggasivakumar/smallocr')
doggasivakumar_modell_path = kagglehub.dataset_download('doggasivakumar/modell')
doggasivakumar_pribe3ec_path = kagglehub.dataset_download('doggasivakumar/pribe3ec')
doggasivakumar_loradapter_path = kagglehub.dataset_download('doggasivakumar/loradapter')
doggasivakumar_0911lorafinetune_path = kagglehub.dataset_download('doggasivakumar/0911lorafinetune')
doggasivakumar_image_test_path = kagglehub.dataset_download('doggasivakumar/image-test')
doggasivakumar_dataset_model_path = kagglehub.dataset_download('doggasivakumar/dataset-model')
doggasivakumar_mathdocument_path = kagglehub.dataset_download('doggasivakumar/mathdocument')

print('Data source import complete.')


In [ ]:
pip install git+https://github.com/boomb0om/CRAFT-text-detection/

In [ ]:
!pip install peft

In [ ]:
# from PIL import Image
# from CRAFT import CRAFTModel, draw_polygons

# model = CRAFTModel('weights/', 'cuda', use_refiner=True, fp16=True)
# img = Image.open('/kaggle/working/converted_image_1.jpeg').convert('RGB')
# polygons = model.get_polygons(img)

In [ ]:
# draw_polygons(img, polygons)

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install jiwer
!pip install datasets
!pip install evaluate
!pip install -U accelerate
!pip install matplotlib
!pip install tensorboard

In [ ]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor
import torch
from PIL import Image
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM

from pdf2image import convert_from_path
import os
import tempfile
from pathlib import Path
import subprocess
import numpy as np
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import cv2
from CRAFT import CRAFTModel, draw_polygons
from fastapi import FastAPI, File, UploadFile,BackgroundTasks
from fastapi.responses import JSONResponse, FileResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
from pdf2image import convert_from_path
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import os
import tempfile
from pathlib import Path
import subprocess
import numpy as np
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch
from PIL import Image
import cv2
from transformers import NougatProcessor, VisionEncoderDecoderModel
import numpy as np
max_length = 100 # defing max length of output

import torch
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType


In [ ]:

# device = xm.xla_device()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Install Poppler

!apt-get install -y poppler-utils

# Install LaTeX
!apt-get install -y texlive texlive-latex-extra texlive-fonts-recommended


In [ ]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-handwritten')
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VisionEncoderDecoderModel.from_pretrained('/kaggle/input/modell/archive/kaggle/working/trocr_handwritten/checkpoint-13540').to(device)
#model = VisionEncoderDecoderModel.from_pretrained('/kaggle/input/smallocr/checkpoint-6770').to(device)
tokenizer = NougatProcessor.from_pretrained(r"CuiSiwei/nougat-for-formula", max_length = max_length) # Replace with your path
feature_extractor = NougatProcessor.from_pretrained(r"CuiSiwei/nougat-for-formula", max_length = max_length) # Replace with your path

mmodel = VisionEncoderDecoderModel.from_pretrained(r"CuiSiwei/nougat-for-formula").to(device) # Replace with your path

lora_config = LoraConfig(

    r=16,

    lora_alpha=16,

    # target_modules=[

    #     # Encoder (Swin Transformer) modules

    #     "attn.qkv",

    #     "attn.proj",

    #     "mlp.fc1",

    #     "mlp.fc2",

    #     # Decoder (GPT-2) modules

    #     "c_attn",

    #     "c_proj",

    #     "c_fc",

    #     "attn.c_proj",

    # ],

    target_modules="all-linear",

    lora_dropout=0.1,

    bias="none"

    # task_type="VL"  # Vision-Language task

)
mmodel = get_peft_model(mmodel,lora_config)
torch.compile(mmodel)
mmodel.merge_and_unload()
mmodel.load_state_dict(torch.load("/kaggle/working/model.pth"))


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify

<All keys matched successfully>

In [ ]:
# # load model, tokenizer, and feature extractor
# mmodel = VisionEncoderDecoderModel.from_pretrained("DGurgurov/im2latex").to(device)
# tokenizer = AutoTokenizer.from_pretrained("DGurgurov/im2latex")
# feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swin-base-patch4-window7-224-in22k") # using the original feature extractor for now
# # tokenizer.pad_token = tokenizer.pad_token


In [ ]:
!pip install python-Levenshtein

In [ ]:

# generate LaTeX formula
# prepare an image
image = cv2.imread("/kaggle/input/dataset-model/mathematical_image.jpg")
orig_h, orig_w = image.shape[:2]

# Scale the image to fit within 224x224 while keeping the aspect ratio
scale = min(224 / orig_w, 224 / orig_h)
new_w, new_h = int(orig_w * scale), int(orig_h * scale)

# Resize the cropped image
resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)

# Create a 224x224 white background with 3 channels if resized is in color
padded = np.ones((224, 224, 3), dtype=np.uint8) * 255 if len(resized.shape) == 3 else np.ones((224, 224), dtype=np.uint8) * 255

# Calculate padding to center the resized image
x_offset = (224 - new_w) // 2
y_offset = (224 - new_h) // 2

# Place the resized image on the white background
padded[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized
pixel_values = feature_extractor(images=padded, return_tensors="pt").pixel_values
pixel_values = pixel_values.to(device)

generated_ids = mmodel.generate(pixel_values)
generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

print("Generated LaTeX formula by model :", generated_texts[0])
# Load the PeftModel correctly and assign it to a variable
loaded_model = PeftModel.from_pretrained(mmodel, "/kaggle/input/0911lorafinetune/checkpoints/checkpoints/checkpoint_step_2000")


# Move pixel_values to the same device as the model
 # Assuming 'device' is defined as 'cuda:0'

# generate LaTeX formula
generated_ids = loaded_model.generate(pixel_values)
generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

print("Generated LaTeX formula by fine tuned model :", generated_texts[0])



In [ ]:
device="cuda"

In [ ]:
from transformers import NougatProcessor, VisionEncoderDecoderModel
from PIL import Image
import cv2
import numpy as np
max_length = 100 # defing max length of output
processor = NougatProcessor.from_pretrained(r"CuiSiwei/nougat-for-formula", max_length = max_length) # Replace with your path
# model = VisionEncoderDecoderModel.from_pretrained(r"CuiSiwei/nougat-for-formula").to(device) # Replace with your path

# generate LaTeX formula
# prepare an image
image = cv2.imread("/kaggle/input/dataset-model/mathematical_image.jpg")
orig_h, orig_w = image.shape[:2]

# Scale the image to fit within 224x224 while keeping the aspect ratio
scale = min(224 / orig_w, 224 / orig_h)
new_w, new_h = int(orig_w * scale), int(orig_h * scale)

# Resize the cropped image
resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)

# Create a 224x224 white background with 3 channels if resized is in color
padded = np.ones((224, 224, 3), dtype=np.uint8) * 255 if len(resized.shape) == 3 else np.ones((224, 224), dtype=np.uint8) * 255

# Calculate padding to center the resized image
x_offset = (224 - new_w) // 2
y_offset = (224 - new_h) // 2

# Place the resized image on the white background
padded[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized
pixel_values = processor(images=padded, return_tensors="pt").pixel_values
pixel_values = pixel_values.to(device)

result_tensor = model.generate(
            pixel_values,
            max_length=max_length,
            bad_words_ids=[[processor.tokenizer.unk_token_id]]
              ) # generate id tensor

result = processor.batch_decode(result_tensor, skip_special_tokens=True) # Using the processor to decode the result
result = processor.post_process_generation(result, fix_markdown=False)

print(*result)


In [ ]:
!pip install python-Levenshtein

In [ ]:
import os
import json

# Step 1: Define the Kaggle credentials dictionary
kaggle_credentials = {"username":"sivadogga","key":"b56da0155f5e3e607c8cffaf048d686c"}

# Step 2: Save the dictionary as kaggle.json
kaggle_json_path = os.path.expanduser("~/.kaggle/kaggle.json")

# Ensure the directory exists
os.makedirs(os.path.dirname(kaggle_json_path), exist_ok=True)

# Write the credentials to the file
with open(kaggle_json_path, "w") as f:
    json.dump(kaggle_credentials, f)

# Set the correct permissions for security
os.chmod(kaggle_json_path, 0o600)



In [ ]:
import os
import json

# Step 1: Define the Kaggle credentials dictionary
kaggle_credentials = {"username":"sivadogga","key":"b56da0155f5e3e607c8cffaf048d686c"}

# Step 2: Save the dictionary as kaggle.json
kaggle_json_path = os.path.expanduser("~/.kaggle/kaggle.json")

# Ensure the directory exists
os.makedirs(os.path.dirname(kaggle_json_path), exist_ok=True)

# Write the credentials to the file
with open(kaggle_json_path, "w") as f:
    json.dump(kaggle_credentials, f)

# Set the correct permissions for security
os.chmod(kaggle_json_path, 0o600)


!kaggle kernels output sivadogga/sharingbe -p /kaggle/working/

*Working code of latex using craft model for word segmentation and text model for extracting text and mathematical equation recognition model*

In [ ]:
polygon_model = CRAFTModel('weights', device , use_refiner=True, fp16=True)
PERMANENT_DIR = "pdf_images"
Path(PERMANENT_DIR).mkdir(parents=True, exist_ok=True)  # Creates the directory if it doesn't exist


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:672: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=No

In [ ]:

import os

import numpy as np
def load_image(image_path):
    """Load an image from a file."""
    return cv2.imread(image_path)


def crop_polygons(image, polygons):
    """
    Crop images based on the provided polygons.
    :param image: The original image.
    :param polygons: List of polygons defining the areas to crop.
    :return: List of cropped images and their y-positions.
    """
    # Convert PIL Image to NumPy array for OpenCV compatibility
    image = np.array(image)


    y_positions = []

    for i, polygon in enumerate(polygons):
        # Create a mask for the polygon
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [np.array(polygon, dtype=np.int32)], 255)

        # Get the bounding box of the polygon
        x, y, w, h = cv2.boundingRect(np.array(polygon, dtype=np.int32))

        # Crop the image using the mask
        masked_image = cv2.bitwise_and(image, image, mask=mask)
        cropped_image = masked_image[y:y+h, x:x+w]

        # Resize the cropped image for the text recognition model


        cropped_images.append(cropped_image)
        y_positions.append(np.mean([point[1] for point in polygon]))  # Mean y-coordinate
        filename = os.path.join('/kaggle/working/pdf_images', f'sytcropped_image_{i + 1}.png')
        cv2.imwrite(filename, cropped_image)
    return cropped_images, y_positions




In [ ]:
max_length=100
def recognize_text_from_images(cropped_images):
    """
    Recognize text from a list of cropped images.
    :param cropped_images: List of cropped images.
    :return: List of recognized text strings.
    """


    for cropped_image in cropped_images:
        # Convert OpenCV BGR image to PIL format for processing
        image = Image.fromarray(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
        pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Process the image and move pixel values to the correct device
        pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values.to(device)


        # if generated_text == "SPECIAL_CHARACTER":
        special_character_images.append(cropped_image)
        generated_ids = mmodel.generate(pixel_values,max_length=max_length,
                    bad_words_ids=[[processor.tokenizer.unk_token_id]])

        generated_tex = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0] # Using the processor to decode the result
        generated_text += tokenizer.post_process_generation(generated_tex, fix_markdown=False)


        recognized_texts.append(generated_text)


    return recognized_texts

**code using carft model for text detection**

In [ ]:
cropped_images = []
special_character_images = []
recognized_texts = []


def upload_pdf(pdf_path):
    _, ext = os.path.splitext(pdf_path)
    ext = ext.lower()
    if ext == ".pdf":
      images = convert_from_path(pdf_path, dpi=200, fmt="jpeg")
    elif ext in [".jpg", ".jpeg", ".png", ".bmp", ".gif"]:
      images = [Image.open(pdf_path)]
    print("setp 2")
    # Save images to the current working directory and collect their paths
    image_paths = []
    for i, image in enumerate(images):
        print("hey")

        print("setp 3")
        if ext == ".pdf":
            filename = os.path.join('/kaggle/working/', f'converted_image_{i + 1}.jpeg')
            image.save(filename, "JPEG")
        image = image.convert('RGB')
        polygons = polygon_model.get_polygons(image)
        result=draw_polygons(image, polygons)
        image_path = os.path.join(f"/kaggle/working/page_{i + 1}.jpeg")
        result.save(image_path, "JPEG")
        image_paths.append(image_path)
            # Crop images and retrieve their y-positions
        cropped_images, y_positions = crop_polygons(image, polygons)
        print(len(cropped_images))
        # Recognize text from cropped images
        recognized_texts = recognize_text_from_images(cropped_images)
        print(len(recognized_texts))
#         recognized_texts = ["sorry will be soon available","facing issues with resources"]
        # Combine recognized texts into paragraphs based on y-positions
        merged_paragraphs = []
        current_paragraph = ""
        threshold = 70  # Define a threshold for merging based on y-coordinate proximity

        for i, text in enumerate(recognized_texts):
            if str(text).strip():  # If the recognized text is not empty
                if current_paragraph:  # If there is already some text in the paragraph
                    # Check if current and last y-position are within the threshold
                    if abs(y_positions[i] - y_positions[i - 1]) < threshold:
                        current_paragraph=  str(text).strip()+ " " + current_paragraph
                    else:
                        merged_paragraphs.append(current_paragraph.strip())
                        current_paragraph = str(text).strip()+ "\\newline"   # Start a new paragraph
                else:
                    current_paragraph = str(text).strip()+ "\\newline"   # Start a new paragraph

        # Append any remaining text
        if current_paragraph:
            merged_paragraphs.append(current_paragraph.strip())

        # Print the recognized text paragraphs
        print("Recognized Text Paragraphs:")
        for i, paragraph in enumerate(merged_paragraphs):
            text+=f"""{paragraph}
"""
        text+= r"""\\newpage"""

    # Minimal LaTeX code for testing
        # Minimal LaTeX code for testing
    header = latex_code = r"""
\documentclass{article}
\usepackage{amsmath}

\begin{document}

\title{Digital Document}
\author{IE 643}
\date{\today}
\maketitle

\begin{verbatim}
"""
    footer=r"""
    \end{verbatim}
    \end{document}"""

    latex_code = header+str(text)+footer
    print(latex_code)
    # Write LaTeX code to file in the current working directory
    latex_file_path = os.path.join(os.getcwd(), "output.tex")
    with open(latex_file_path, "w") as f:
        f.write(latex_code)

    # Compile the LaTeX file
    process = subprocess.run(['pdflatex', latex_file_path], capture_output=True, text=True)

    if process.returncode != 0:
        print("LaTeX compilation failed:")
        print(process.stderr)

    else:
        print("LaTeX compilation success:")
        pdf_output_path = os.path.join(os.getcwd(), "ie643_output.pdf")




def cleanup_files(file_paths):
    """Delete files from provided paths."""
    for file_path in file_paths:
        try:
            if os.path.exists(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}: {e}")


upload_pdf("/kaggle/input/mathdocument/handwritten_mathematical.pdf")

setp 2
hey
setp 3
8
8
Recognized Text Paragraphs:

\documentclass{article}
\usepackage{amsmath}

\begin{document}

\title{Digital Document}
\author{IE 643}
\date{\today}
\maketitle

\begin{verbatim}
i-1\tilde{t}_{1}

The formula for\Pi_{k}f_{m}\wedge_{k}f_{m} cross-endroy\overline{cos\theta_{S}}=arctan\varphi_{P}\varphi_{P}\overline{\varphi} loss canlog_{S}tan be\mathbb{Z}\newline
writings.\overline{|uncitive}\newline
SPECIAL_CHARACTER\overline{L(y,\hat{y})}= -Eyi.logy-\sum^{i}y_{i}\cdot log\hat{y}_{i}\newline
i-1\tilde{t}_{1}\newline
\\newpage
    \end{verbatim}
    \end{document}
LaTeX compilation success:


**code using latest cv2 version to detect words in an image**

In [ ]:
cropped_images = []
special_character_images = []
recognized_texts = []
def upload_pdf(pdf_path):
    _, ext = os.path.splitext(pdf_path)
    ext = ext.lower()
    if ext == ".pdf":
      images = convert_from_path(pdf_path, dpi=200, fmt="jpeg")
    elif ext in [".jpg", ".jpeg", ".png", ".bmp", ".gif"]:
      images = [cv2.imread(pdf_path)]
    print("setp 2")
    # Save images to the current working directory and collect their paths
    image_paths = []
    merged_paragraphs = []
     #  initialize empty list
    full_text=""
    for i, image in enumerate(images):

        print("setp 3")
        if ext == ".pdf":
            filename = os.path.join('/kaggle/working/', f'converted_image_{i + 1}.jpeg')
            image.save(filename, "JPEG")
            img = cv2.imread(filename)
            image=img

             #  Creating a copy of the image to make changes
            grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  #  Converting to greyscale
            # Apply Otsu filtering
            otsu1 = cv2.threshold(grey, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)[1]
#             horz_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (14, 1))
#             # The above line creates a rectangular kernel of dimension (14, 1)
#             lines = cv2.morphologyEx(otsu1, cv2.MORPH_OPEN, horz_kernel, iterations=2)

#             cnts = cv2.findContours(lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # findContours returns the contours and the hierarchy for opencv version 4.0 and later
            # earlier versions return: [image, contours, hierarchy]
#             cnts = cnts[0] if len(cnts) == 2 else cnts[1]
#             for c in cnts:
#                 cv2.drawContours(img, [c], -1, (255, 255, 255), 2)
            grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            otsu2 = cv2.threshold(grey, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)[1]
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (64,9))
            dilation = cv2.dilate(otsu2, kernel, iterations = 1)
            contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                                        cv2.CHAIN_APPROX_NONE)

            count = 0  #  keep a count for the words (optional)


            y_positions=[]
            for c in contours:
                x, y, w, h = cv2.boundingRect(c)  #  get coordinates of contouts
                if w>50 and h>25:  #  threshold for size of bounding box
                    count += 1
                    cropped = image[y:y+h, x:x+w]  # Crop the image

                    # Calculate scale to fit the larger side into 224 while keeping aspect ratio
                    scale = min(224 / w, 224 / h)
                    new_w, new_h = int(w * scale), int(h * scale)

                    # Resize the cropped image
                    resized = cv2.resize(cropped, (new_w, new_h), interpolation=cv2.INTER_AREA)

                    # Create a 224x224 white background
                    padded = np.ones((224, 224, 3), dtype=np.uint8) * 255 if len(resized.shape) == 3 else np.ones((224, 224), dtype=np.uint8) * 255

                    # Calculate padding to center the image
                    x_offset = (224 - new_w) // 2
                    y_offset = (224 - new_h) // 2

                    # Place the resized image on the white background
                    padded[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized

                    # Append to the list
                    cropped_images.append(padded)
            #         cropped_images.append(cv2.resize(otsu2[y:h+y, x:w+x], (224, 224))) # Resizing to fixed size
#                     rect = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) # Draw Bounding boxes
                    y_positions.append(np.mean([y,y+h]))

            print(count, " words extracted!")     # Recognize text from cropped images
            recognized_texts = recognize_text_from_images(cropped_images)
            print(len(recognized_texts))
    #         recognized_texts = ["sorry will be soon available","facing issues with resources"]
            # Combine recognized texts into paragraphs based on y-positions

            current_paragraph = ""
            threshold = 70  # Define a threshold for merging based on y-coordinate proximity

            for i, text in enumerate(recognized_texts):
                if text.strip():  # If the recognized text is not empty
                    if current_paragraph:  # If there is already some text in the paragraph
                        # Check if current and last y-position are within the threshold
                        if abs(y_positions[i] - y_positions[i - 1]) < threshold:
                            current_paragraph=  current_paragraph + text.strip()+ " "
                        else:
                            merged_paragraphs.append(current_paragraph.strip())
                            current_paragraph = text.strip()+ " "   # Start a new paragraph
                    else:
                        current_paragraph = text.strip()+ " "   # Start a new paragraph

            # Append any remaining text
            if current_paragraph:
                merged_paragraphs.append(current_paragraph.strip())
        else:
            cropped_img=image
            orig_h, orig_w = cropped_img.shape[:2]

            # Scale the image to fit within 224x224 while keeping the aspect ratio
            scale = min(224 / orig_w, 224 / orig_h)
            new_w, new_h = int(orig_w * scale), int(orig_h * scale)

            # Resize the cropped image
            resized = cv2.resize(cropped_img, (new_w, new_h), interpolation=cv2.INTER_AREA)

            # Create a 224x224 white background with 3 channels if resized is in color
            padded = np.ones((224, 224, 3), dtype=np.uint8) * 255 if len(resized.shape) == 3 else np.ones((224, 224), dtype=np.uint8) * 255

            # Calculate padding to center the resized image
            x_offset = (224 - new_w) // 2
            y_offset = (224 - new_h) // 2

            # Place the resized image on the white background
            padded[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized

            # Append to the list of cropped images
            cropped_images.append(padded)

            # Recognize text from the padded image and store in merged_paragraphs
            merged_paragraphs.append(recognize_text_from_images([padded]))

        # Print the recognized text paragraphs
        print("Recognized Text Paragraphs:")
        for i, paragraph in enumerate(merged_paragraphs):
            full_text+=f"""{paragraph}
"""
        full_text+= r""" """

    # Minimal LaTeX code for testing
        # Minimal LaTeX code for testing
    header = latex_code = r"""
\documentclass{article}
\usepackage{amsmath}

\begin{document}

\title{Digital Document}
\author{IE 643}
\date{\today}
\maketitle

\begin{verbatim}
"""
    footer=r"""
    \end{verbatim}
    \end{document}"""

    latex_code = header+full_text+footer
    print(latex_code)
    # Write LaTeX code to file in the current working directory
    latex_file_path = os.path.join(os.getcwd(), "output.tex")
    with open(latex_file_path, "w") as f:
        f.write(latex_code)

    # Compile the LaTeX file
    process = subprocess.run(['pdflatex', latex_file_path], capture_output=True, text=True)

    if process.returncode != 0:
        print("LaTeX compilation failed:")
        print(process.stderr)

    else:
        print("LaTeX compilation success:")
        pdf_output_path = os.path.join(os.getcwd(), "output.pdf")



upload_pdf("/kaggle/input/mathdocument/handwritten_mathematical.pdf")

setp 2
setp 3
10  words extracted!
10
Recognized Text Paragraphs:

\documentclass{article}
\usepackage{amsmath}

\begin{document}

\title{Digital Document}
\author{IE 643}
\date{\today}
\maketitle

\begin{verbatim}
\tilde{t}_{1}
L(y,\hat{y})= -\overline{\zeta}^{1} \hat{y}_{i}\cdotlog\hat{y}_{i}
\hat{a}_{a}
uncible_{n}a_{3}:
\tilde{\gamma} \pi_{e}f_{sim}^{f_{sim}} cos-entropy^{log}cosan \mathbb{Z}
 
    \end{verbatim}
    \end{document}
LaTeX compilation success:


*end of the code*


In [ ]:
len(cropped_images)

In [ ]:
import matplotlib.pyplot as plt

# Define grid size for displaying all 162 images
num_images = len(special_character_images)
rows = 9
cols = 18

# Set up the figure size to accommodate the grid
plt.figure(figsize=(20, 20))

# Loop through each cropped and padded image
for i in range(num_images):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(special_character_images[i],cmap='gray')  # Display each image in grayscale

    plt.axis('off')  # Turn off axis for a cleaner look

# Adjust layout to minimize gaps between subplots
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Define grid size for displaying all 162 images
num_images = len(cropped_images)
rows = 9
cols = 18

# Set up the figure size to accommodate the grid
plt.figure(figsize=(20, 20))

# Loop through each cropped and padded image
for i in range(num_images):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(cropped_images[i])  # Display each image in grayscale
    plt.title(recognized_texts[i])
    plt.axis('off')  # Turn off axis for a cleaner look

# Adjust layout to minimize gaps between subplots
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
ax = plt.subplots(2,3, figsize=(12,7))[1]
ax[0,0].imshow(image)
ax[0,0].set_title('Image')

ax[0,2].imshow(img)
ax[0,2].set_title('Lines Removed')
ax[1,0].imshow(otsu2)
ax[1,0].set_title('Re-Otsuing')
ax[1,1].imshow(dilation)
ax[1,1].set_title('Dilated')
ax[1,2].imshow(image)
ax[1,2].set_title('Result')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the final result in a larger size
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title('Final Result (Larger View)')
plt.axis('off')
plt.show()



In [ ]:
img = cv2.imread(r"/kaggle/input/dataset-model/20241107_185154.jpg")
image = img.copy()  #  Creating a copy of the image to make changes
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  #  Converting to greyscale
# Apply Otsu filtering
otsu1 = cv2.threshold(grey, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)[1]
horz_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (14, 1))
# The above line creates a rectangular kernel of dimension (14, 1)
lines = cv2.morphologyEx(otsu1, cv2.MORPH_OPEN, horz_kernel, iterations=2)

cnts = cv2.findContours(lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# findContours returns the contours and the hierarchy for opencv version 4.0 and later
# earlier versions return: [image, contours, hierarchy]
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(img, [c], -1, (255, 255, 255), 2)
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
otsu2 = cv2.threshold(grey, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (14,9))
dilation = cv2.dilate(otsu2, kernel, iterations = 1)
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_NONE)

img_list = []  #  initialize empty list
count = 0  #  keep a count for the words (optional)
cropped_images = []  #  initialize empty list
count = 0  #  keep a count for the words (optional)
y_positions=[]
for c in contours:
    x, y, w, h = cv2.boundingRect(c)  #  get coordinates of contouts
    if w>50 and h>25:  #  threshold for size of bounding box
        count += 1
        cropped = otsu2[y:y+h, x:x+w]  # Crop the image

        # Calculate scale to fit the larger side into 224 while keeping aspect ratio
        scale = min(224 / w, 224 / h)
        new_w, new_h = int(w * scale), int(h * scale)

        # Resize the cropped image
        resized = cv2.resize(cropped, (new_w, new_h), interpolation=cv2.INTER_AREA)

        # Create a 224x224 white background
        padded = np.ones((224, 224), dtype=np.uint8) * 255

        # Calculate padding to center the image
        x_offset = (224 - new_w) // 2
        y_offset = (224 - new_h) // 2

        # Place the resized image on the white background
        padded[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized

        # Append to the list
        cropped_images.append(padded)
#         cropped_images.append(cv2.resize(otsu2[y:h+y, x:w+x], (224, 224))) # Resizing to fixed size
        rect = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) # Draw Bounding boxes
        y_positions.append(np.mean([y,y+h]))

print(count, " words extracted!")